In [96]:
# This notebook is created to document and help me with the development process - isn't required to run the pipeline

# GIF: Nicolas Cage Montage
---
Combine a sequence of images of Nicolas Cage's face, sorted by smartgrid

In [9]:
# Goal ====================
# Directories:
    # temp - temp files
    # inputs - source images - 
    # outputs - processed images in sequence
    # movies - final movie gif
#  Scripts:
    # build-directories.sh - script to create folders
    # run-cage.sh - used to run the pipeline as outlined below
        # get-cage.sh - scrapes images of nicolas cage and puts them into inputs
        # create-cage-bounds.py - script to detect coordinates of nick cage faces in photos - exports to a csv
        # crop-cage.sh - goes through input images and crops cage faces as per exported csv
        # sort-cage.sh - use smartgrid to sort images and put them into outputs
        # composite-cage - use smartgrid to sort images and put them into outputs    

In [28]:
# Build Directories ===================
build_dirs = 'mkdir temp inputs outputs movies'
! {build_dirs}

mkdir: temp: File exists
mkdir: inputs: File exists
mkdir: outputs: File exists
mkdir: movies: File exists


In [5]:
# Misc. helpers
def write_script(title, content) :
    f = open('{}.sh'.format(title), 'w')
    f.write(content)
    f.close()

## Download Nick Cage Photos

In [92]:
# Get Nick Cage photos ================
# source: http://niccageaseveryone.blogspot.co.nz
get_cage = '\
set -ex\
\nfor YEAR in 2009 2010 2011 2013\
\ndo\
\n\tfor PAGE in $(seq -f "%02g" 1 12)\
\n\tdo\
\n\t# build the url \
\n\tURL="http://niccageaseveryone.blogspot.co.nz/"$YEAR"/"$PAGE\
\n\t\t# fetch the images \\\
\n\t\twget \\\
\n\t\t\t--adjust-extension \\\
\n\t\t\t--no-directories \\\
\n\t\t\t--convert-links \\\
\n\t\t\t--backup-converted \\\
\n\t\t\t--random-wait\\\
\n\t\t\t--limit-rate=100k \\\
\n\t\t\t--span-hosts \\\
\n\t\t\t--directory-prefix=temp/downloads \\\
\n\t\t\t--page-requisites \\\
\n\t\t\t--timestamping \\\
\n\t\t\t--execute robots=off \\\
\n\t\t\t--accept=*.jpg \\\
\n\t\t\t$URL\
\n\tdone\
\ndone'

write_script("get-cage", get_cage)

## Process Nick Cage Photos

In [95]:
# Process Nick Cage photos ================
# Detect Nick Cage faces - filter out non cage faces + crop

# Use python face recognition library to create a csv of cage bounding boxes =========
"""
import face_recognition
import os

# create encoding of known cage image to compare faces with
cage_image = face_recognition.load_image_file('./temp/face_recognition/to_recognise/Nicolas Cage.jpg')
cage_encoding = face_recognition.face_encodings(cage_image)[0]

# loop through downloaded images - create csv of cage face locations for cropping shell script to refer to
cage_csv = open('./temp/face_recognition/cage_bounds.csv', 'w')
column_titles = 'file_name, face_index, top, right, bottom, left'
cage_csv.write(column_titles + '\n')

for file in os.listdir('./temp/downloads'): 
    if file.endswith('.jpg'):
        image_path = os.path.join('./temp/downloads', file)
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.api.face_locations(image)
        if (len(face_locations) > 0): # if faces are detected
            # encode each face
            image_face_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)
            cage_locations = [] # list of bounding box coordinates for cage faces
            for index, encoding in enumerate(image_face_encodings):
                results = face_recognition.compare_faces([cage_encoding], encoding, tolerance=0.8)
                if (results[0] == True):
                    bounds = face_locations[index]
                    cage_csv.write('{},{},{},{},{},{}\n'.format(file, index, bounds[0], bounds[1], bounds[2], bounds[3]))
"""

"\nimport face_recognition\nimport os\n\n# create encoding of known cage image to compare faces with\ncage_image = face_recognition.load_image_file('./temp/face_recognition/to_recognise/Nicolas Cage.jpg')\ncage_encoding = face_recognition.face_encodings(cage_image)[0]\n\n# loop through downloaded images - create csv of cage face locations for cropping shell script to refer to\ncage_csv = open('./temp/face_recognition/cage_bounds.csv', 'w')\ncolumn_titles = 'file_name, face_index, top, right, bottom, left'\ncage_csv.write(column_titles + '\n')\n\nfor file in os.listdir('./temp/downloads'): \n    if file.endswith('.jpg'):\n        image_path = os.path.join('./temp/downloads', file)\n        image = face_recognition.load_image_file(image_path)\n        face_locations = face_recognition.api.face_locations(image)\n        if (len(face_locations) > 0): # if faces are detected\n            # encode each face\n            image_face_encodings = face_recognition.face_encodings(image, known_face

In [12]:
# Shell script and imagemagick to iterate through CSV and create folder of cropped images
    # Image logic
        # Get width of face
        # Get ratio of it compared to a desired rescaled size, resize by that factor
        # Calculate $LEFT, $TOP crop dimensions required to center the face

crop_cage = '\
while IFS=, read -a line;\
\ndo\
\n\tINPUTPATH=./temp/downloads/${line[0]}\
\n\tOUTPUTPATH=./temp/cropped_cage/${line[0]}-${line[1]}.jpg\
\n\tFACEWIDTH=$((${line[3]}-${line[5]}))\
\n\tFACEHEIGHT=$((${line[4]}-${line[2]}))\
\n\tRESCALE_FACTOR=$(bc <<< "scale=2;(220/${FACEWIDTH})*100")\
\n\tSCALED_TOP=$(bc <<< "scale=2;${line[2]}*${RESCALE_FACTOR}/100")\
\n\tSCALED_LEFT=$(bc <<< "scale=2;${line[5]}*${RESCALE_FACTOR}/100")\
\n\tconvert ${INPUTPATH} -crop ${FACEWIDTH}x${FACEHEIGHT}+${line[5]}+${line[2]} +repage -resize ${RESCALE_FACTOR}% ${OUTPUTPATH}\
\ndone < ./temp/face_recognition/cage_bounds.csv\
'

write_script('crop_cage', crop_cage)

In [57]:
# Shell script to sort cropped Nick Cage faces
sort_cage = "\
set -ex\
\n\
python /usr/local/anaconda/extras/smartgrid.py\ \
  --use-imagemagick\ \
  --input-glob 'temp/cropped_cage/*'\ \
  --aspect-ratio 1.92\ \
  --drop-to-fit\ \
  --output-path 'temp/smartgrid'\
"
write_script('sort_cage', sort_cage)

In [142]:
# # Create sequence of images from sorted images in csv
# import numpy
# file = open('temp/smartgrid/montage_26x13.txt', 'r')

# # 26 x 13 grid of faces - get indexes of images in a snakey order (instead of going from top to bottom repeatedly, it alternates between going from top to bottom and vice versa - more order)
# def get_snake_loop(arr, snake_direction, grid_x, grid_y):
#     snake_list=[]
#     if (snake_direction == 'x'):
#         split_arr = numpy.array_split(numpy.array(arr),grid_y)
#     elif (snake_direction == 'y'):
#         split_arr = numpy.array_split(numpy.array(arr),grid_x)
#     index = 0
#     for i in split_arr:
#         to_flatten=[]
#         if (index%2 == 0):
#             to_flatten = i
#         else:
#             to_flatten = i[::-1]
#         for val in to_flatten:
#             snake_list.append(val)
#         index += 1
#     return snake_list

# imgs = []
# for line in file:
#     imgs.append(line.replace('"','').strip())

# x_sequence = get_snake_loop(imgs, 'x', 23, 16)
# y_sequence = get_snake_loop(imgs, 'y', 23, 16)

# # Create sequenced list of images in csv file ======================
# sequence_csv = open('./temp/sequencer/sequence.csv', 'w')
# column_titles = 'index,x_img,y_img'
# sequence_csv.write(column_titles + '\n')

# index = 0
# for i in range(len(x_sequence)):
#     sequence_csv.write('{},{},{}\n'.format(index, x_sequence[index], y_sequence[index]))
#     index+=1

# sequence_csv.close()

In [140]:
# Shell script to compose image sequence for gifs
compose_cage = "\
while read p;\
\ndo\
\n\techo $p\
\ndone <./temp/sequencer/x_sequence.txt"

! {compose_cage}

temp/cropped_cage/Bernie.jpg-1.jpg
temp/cropped_cage/ALSTRAGGASNICKBAKERnic+cage+who_.jpg-1.jpg
temp/cropped_cage/DanielShealeyNCaTT.jpg-0.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-6.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-10.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-2.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-8.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-1.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-0.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-5.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-11.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-4.jpg
temp/cropped_cage/Adrienne+Rule+-+molecule-adamantane.jpg-7.jpg
temp/cropped_cage/AVATIZER4268258978_15abb01cc3.jpg-0.jpg
temp/cropped_cage/NICHOLASTORRESCageBeingInterviewedByCageAsLetterman.jpg-0.jpg
temp/cropped_cage/PAULCUMMINSNic+Cage+as+Austin+Powers.jpg-0.jpg
temp/cropped